In [3]:
pd.set_option('display.width', 1000)

In [ ]:
# ===========================
# Investigated the homogeneous binary nucleation of atmospherically-relevant molecules
# =========================== 

# 🧱 Generate Dataframe: 
#     Include mole-fraction data 
# ---------------------------
# Table of Contents
# 1️⃣ Compute mole-fraction 
# 2️⃣ Review generated parquets

In [1]:
# ===============================
# 1️⃣ Compute mole-fraction 
# ===============================
# 📆 Date created: October 12, 2024 
# 📆 Date updated: October 13, 2024 
# - reads parquet batch files

import pandas as pd
import numpy as np
import glob
import pyarrow.parquet as pq
import pyarrow as pa

# Mapping of molecule names to number of atoms per molecule
molecule_atoms = {
    'SOL': 4,
    'NON': 9,
    'BUTA': 6,
    'MET': 3
}

# Define the priority order for molecule assignment
molecule_priority = ['SOL', 'NON', 'BUTA', 'MET']

# Get the list of batch Parquet files produced by the previous script
batch_files = sorted(glob.glob('nuc_fort-to-panda_batch_*.parquet'), key=lambda x: int(x.split('_')[-1].split('.')[0]))

# Check if any batch files were found
if not batch_files:
    print("No batch Parquet files found matching the pattern 'nuc_fort-to-panda_batch_*.parquet'.")
else:
    print(f"Batch files found: {batch_files}")
    
    # List to keep track of result batch files
    result_batch_files = []
    
    batch_number = 1  # Initialize batch number
    
    # Process each batch file
    for batch_file in batch_files:
        print(f"\nProcessing batch file: {batch_file}")
        
        # Read the batch Parquet file
        df = pd.read_parquet(batch_file)
        
        # Determine the molecules present in the dataset
        molecules_present = df['molecule_name'].unique()
        
        # Intersect the priority list with the molecules present to maintain priority order
        molecules_in_priority = [mol for mol in molecule_priority if mol in molecules_present]
        
        # Ensure we have at least two molecules to assign
        if len(molecules_in_priority) >= 2:
            molecule1 = molecules_in_priority[0]
            molecule2 = molecules_in_priority[1]
        else:
            # If less than two molecules are present, fill the remaining with None or the next in priority
            molecule1 = molecules_in_priority[0] if len(molecules_in_priority) >= 1 else None
            molecule2 = None
            print("Less than two molecules present in the dataset. Molecule2 will be set to None.")
        
        print(f"Assigned molecule1: {molecule1}")
        print(f"Assigned molecule2: {molecule2}")
        
        # Proceed with the analysis only if molecule1 is assigned
        if molecule1 is None:
            print("No molecules found in the dataset. Skipping this batch.")
            continue
        else:
            # Filter for nuclei with size greater than 1
            df_filtered = df[df['size'] > 1]
        
            # Group the data by 'frame_number' and 'nucleus_id'
            grouped = df_filtered.groupby(['frame_number', 'nucleus_id'])
        
            # Initialize a list to store the results
            results = []
        
            # Iterate over each group (each nucleus in each frame)
            for (frame_number, nucleus_id), group in grouped:
                size = group['size'].iloc[0]  # All rows in the group have the same size
                molecule_counts = {}
        
                # Initialize counts for molecule1 and molecule2 to zero
                molecule_counts[molecule1] = 0
                if molecule2 is not None:
                    molecule_counts[molecule2] = 0
        
                # Determine the counts of molecules in the nucleus
                for mol in molecule_counts.keys():
                    # Get the number of atoms of this molecule in the nucleus
                    num_atoms = group[group['molecule_name'] == mol].shape[0]
                    num_atoms_per_molecule = molecule_atoms.get(mol)
                    if num_atoms_per_molecule is None:
                        print(f"Unknown molecule '{mol}' encountered. Please update molecule_atoms mapping.")
                        continue  # Skip unknown molecules
                    num_molecules = num_atoms / num_atoms_per_molecule
                    molecule_counts[mol] = num_molecules
        
                count1 = molecule_counts[molecule1]
                count2 = molecule_counts.get(molecule2, 0)
                total_molecules = count1 + count2
        
                # Calculate mole fractions
                if total_molecules != 0:
                    mole_fraction_1 = count1 / total_molecules
                    mole_fraction_2 = count2 / total_molecules
                else:
                    mole_fraction_1 = np.nan
                    mole_fraction_2 = np.nan
        
                # Append the results to the list
                results.append({
                    'frame': frame_number,
                    'size': size,
                    'nucleus_id': nucleus_id,
                    'mole_fraction_molecule1': mole_fraction_1,
                    'mole_fraction_molecule2': mole_fraction_2,
                    'molecule1': molecule1,
                    'molecule2': molecule2
                })
        
            # Create a new DataFrame from the results
            df_results = pd.DataFrame(results)
        
            # Define the result batch filename using the same naming convention
            result_batch_filename = f'nuc_analysis_mole-fraction_batch_{batch_number}.parquet'
            result_batch_files.append(result_batch_filename)
        
            # Save the results to a Parquet file
            df_results.to_parquet(result_batch_filename, index=False)
            print(f"Batch {batch_number}: Results saved to '{result_batch_filename}'.")
        
        batch_number += 1  # Increment batch number
    
    # After processing all batches, combine the result batch Parquet files
    print("\nCombining result batch files into 'nuc_analysis_mole-fraction_combined.parquet'...")
    
    # Read all result batch Parquet files and append to a list
    result_tables = []
    for result_batch_file in result_batch_files:
        table = pq.read_table(result_batch_file)
        result_tables.append(table)
        print(f"Added '{result_batch_file}' to the combined Parquet file.")
    
    # Concatenate all tables
    combined_result_table = pa.concat_tables(result_tables)
    
    # Write the combined table to the output Parquet file
    combined_output_filename = 'nuc_analysis_mole-fraction_combined.parquet'
    pq.write_table(combined_result_table, combined_output_filename)
    print(f"\nAll results have been written to '{combined_output_filename}'.")
    print(f"Individual result batch Parquet files have been kept.")


Batch files found: ['nuc_fort-to-panda_batch_1.parquet', 'nuc_fort-to-panda_batch_2.parquet', 'nuc_fort-to-panda_batch_3.parquet', 'nuc_fort-to-panda_batch_4.parquet', 'nuc_fort-to-panda_batch_5.parquet', 'nuc_fort-to-panda_batch_6.parquet', 'nuc_fort-to-panda_batch_7.parquet', 'nuc_fort-to-panda_batch_8.parquet', 'nuc_fort-to-panda_batch_9.parquet', 'nuc_fort-to-panda_batch_10.parquet', 'nuc_fort-to-panda_batch_11.parquet', 'nuc_fort-to-panda_batch_12.parquet', 'nuc_fort-to-panda_batch_13.parquet', 'nuc_fort-to-panda_batch_14.parquet', 'nuc_fort-to-panda_batch_15.parquet', 'nuc_fort-to-panda_batch_16.parquet', 'nuc_fort-to-panda_batch_17.parquet', 'nuc_fort-to-panda_batch_18.parquet', 'nuc_fort-to-panda_batch_19.parquet', 'nuc_fort-to-panda_batch_20.parquet', 'nuc_fort-to-panda_batch_21.parquet']

Processing batch file: nuc_fort-to-panda_batch_1.parquet
Assigned molecule1: BUTA
Assigned molecule2: MET
Batch 1: Results saved to 'nuc_analysis_mole-fraction_batch_1.parquet'.

Processing 

In [6]:
# ===============================
# 2️⃣ Review parquets
# ===============================
# 📆 Date created: October 12, 2024 

# Remove wrapping
pd.set_option('display.width', 1000)

# Upload df
df_combined = pd.read_parquet('nuc_analysis_mole-fraction_combined.parquet')

# Display the first few rows
print(" ")
print(df_combined.head())

print(" ")
print("--> Info of df under inspection:")
print(" ")
print(df_combined.info())

print(" ") 
print("--> Duplicate rows identified:")
print(" ")
duplicate = df_combined[df_combined.duplicated()]
print(duplicate)

print(" ")
print("--> NaN identified:")
print(" ")
print(df_combined.isna().sum())

 
   frame  size  nucleus_id  mole_fraction_molecule1  mole_fraction_molecule2 molecule1 molecule2
0      0     3          11                 1.000000                 0.000000      BUTA       MET
1      0     2          13                 1.000000                 0.000000      BUTA       MET
2      0     4          25                 0.500000                 0.500000      BUTA       MET
3      0     3          26                 0.333333                 0.666667      BUTA       MET
4      0     2          38                 0.500000                 0.500000      BUTA       MET
 
--> Info of df under inspection:
 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3281 entries, 0 to 3280
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   frame                    3281 non-null   int64  
 1   size                     3281 non-null   int64  
 2   nucleus_id               3281 non-null   int64  
 3 